In [ ]:
!pip install transformers diffusers accelerate fastapi uvicorn nest-asynci

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_unet.safetensors" # Use the correct ckpt for your step setting!

# Load model.
unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cuda", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cuda"))
pipe = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant="fp16").to("cuda")

# Ensure sampler uses "trailing" timesteps.
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

In [ ]:
 

from fastapi import FastAPI, Request
from fastapi.responses import FileResponse
from pyngrok import ngrok
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import nest_asyncio
import uvicorn
import uuid
import os
from google.colab import userdata
from fastapi.responses import StreamingResponse
from io import BytesIO

NGROK_AUTH_TOKEN=userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_AUTH_TOKEN) # Replace with your actual authtoken


app = FastAPI()
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)
@app.post("/generatenew")
async def generate(request: Request):
    data = await request.json()
    prompt = data.get("prompt", "A cat sitting on a rocket")
    image = pipe(prompt).images[0]
    
    # Convert the image to bytes in PNG format
    image_bytes = BytesIO()
    image.save(image_bytes, format='PNG')
    image_bytes.seek(0)
    
    return StreamingResponse(image_bytes, media_type="image/png")
 
@app.post("/generate")
async def generate(request: Request):
    data = await request.json()
    prompt = data.get("prompt", "A cat sitting on a rocket")
    image = pipe(prompt).images[0]
    print(f'image:{image}')
    image_path = f"/content/{arman.hex}.png"
    image.save(image_path)
    return {"image_url": ngrok_tunnel.public_url + "/image/" + os.path.basename(image_path),"image":image}

@app.get("/image/{img_name}")
def serve_image(img_name: str):
    path = f"/content/{img_name}"
    return FileResponse(path, media_type="image/png")

nest_asyncio.apply()
uvicorn.run(app, port=8000)


In [ ]:
user_img_prompt = """Create a detailed, photorealistic image of the following scene:\n        Leo spots a lion.\n        \n        **Main Characters**:\n        Leo - A young boy with bright eyes and short brown hair, wearing a simple, slightly torn shirt and shorts., Curious, adventurous, a little bit scared but brave.\n \n\n   """  
resp=pipe(user_img_prompt,negative_prompt="anime, illustration", num_inference_steps=8, guidance_scale=0).images[0] 
display(resp)